### 피처 
IRS 1년물, 10년물/ CRS 1년물, NDF 1월물, 국고채 1년물, 통안채 1년물, 스왑포인트

- IRS 1년물, 10년물 선정이유: 히트맵에서 IRS 1,2,3,5,10년물간의 상관관계를 봤을때  1,2년물/ 3,5,10년물끼리 다중공선성이 나와서 피처로 2개를 사용해도 상관없다고 판다

### 환율예측 회귀식 
Yt = Yt-1 + MDF 변화량 + 스왑포인트 변화량 + 스왑베이시스 1년물 변화율 + 국고채 1년물 변화율 + 통안채 1년물 변화율

In [2]:
import pandas as pd
import numpy as np

# 파일 불러오기
all = pd.read_excel("./xlsx/Join_data.xlsx", index_col = 0)              # index_col = 0 파일 읽어올 떄 "Unnamed: 0" 컬럼이 추가돼서 해당 컬럼 제거

# 컬럼 추출
all2 = all[['DateTime', '종가_ex','국고1년', '국고3년', '국고5년',
       '국고10년', '통안364일', '통안2년', 'Mid_ndf',
       '1Y_베이시스', '2Y_베이시스', '3Y_베이시스', '5Y_베이시스', '10Y_베이시스', 'M1_스왑포인트']]            # [[]] 대괄호 2개 사용 -> 데이터 프레임형태로 나옴

# 0값 삭제
index2 = all2[all2["통안364일"] == 0].index
all2.drop(index2, inplace=True)
all2.reset_index(drop=True, inplace=True)

# 대비 계산
all2['대비_국고1년'] = all2['국고1년']-all2['국고1년'].shift(1)
all2['대비_국고3년'] = all2['국고3년']-all2['국고3년'].shift(1)
all2['대비_국고5년'] = all2['국고5년']-all2['국고5년'].shift(1)
all2['대비_국고10년'] = all2['국고10년']-all2['국고10년'].shift(1)
all2['대비_통안1년'] = all2['통안364일']-all2['통안364일'].shift(1)
all2['대비_통안2년'] = all2['통안364일']-all2['통안364일'].shift(1)
all2['대비_베이시스1년'] = all2['1Y_베이시스']-all2['1Y_베이시스'].shift(1)
all2['대비_베이시스2년'] = all2['2Y_베이시스']-all2['2Y_베이시스'].shift(1)
all2['대비_베이시스3년'] = all2['3Y_베이시스']-all2['3Y_베이시스'].shift(1)
all2['대비_베이시스5년'] =all2['5Y_베이시스']-all2['5Y_베이시스'].shift(1)
all2['대비_베이시스10년'] = all2['10Y_베이시스']-all2['10Y_베이시스'].shift(1)
all2['대비_ndf'] = all2['Mid_ndf']-all2['Mid_ndf'].shift(1)
all2['스왑포인트_1월물'] = all2["M1_스왑포인트"]/100 

# 전일종가 계산
all2['전일종가_ex'] = all2['종가_ex'].shift(1)

# Nan 행 지우고 인덱스 초기화
all2.drop([0], axis=0, inplace=True)
all2.reset_index(drop=True, inplace=True)

# 필요 칼럼만 남기기
all3 = all2.copy()
all3 = all2[["DateTime", "종가_ex", "전일종가_ex", "대비_국고1년", "대비_국고3년", "대비_국고5년", "대비_국고10년", "대비_통안1년", "대비_통안2년", "대비_베이시스1년", "대비_베이시스2년", "대비_베이시스3년", "대비_베이시스5년","대비_베이시스10년","대비_ndf", "스왑포인트_1월물"]]

# DateTime을 인덱스로 바꿔주기
all4 = all3.set_index('DateTime')
all4.head(2)

C:\Users\hyeok\AppData\Local\Temp\ipykernel_30728\145170077.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all2.drop(index2, inplace=True)
C:\Users\hyeok\AppData\Local\Temp\ipykernel_30728\145170077.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all2['대비_국고1년'] = all2['국고1년']-all2['국고1년'].shift(1)
C:\Users\hyeok\AppData\Local\Temp\ipykernel_30728\145170077.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

,종가_ex,전일종가_ex,대비_국고1년,대비_국고3년,대비_국고5년,대비_국고10년,대비_통안1년,대비_통안2년,대비_베이시스1년,대비_베이시스2년,대비_베이시스3년,대비_베이시스5년,대비_베이시스10년,대비_ndf,스왑포인트_1월물
DateTime,,,,,,,,,,,,,,,
2012-08-02,1131.7,1126.5,-0.03,-0.04,-0.04,-0.04,-0.03,-0.03,2.0,8.0,9.0,9.0,9.0,1.75,2.5
2012-08-03,1134.8,1131.7,-0.02,-0.02,-0.07,-0.07,-0.02,-0.02,2.0,1.5,1.0,-5.0,-13.0,4.00,2.4


In [3]:
all4

,종가_ex,전일종가_ex,대비_국고1년,대비_국고3년,대비_국고5년,대비_국고10년,대비_통안1년,대비_통안2년,대비_베이시스1년,대비_베이시스2년,대비_베이시스3년,대비_베이시스5년,대비_베이시스10년,대비_ndf,스왑포인트_1월물
DateTime,,,,,,,,,,,,,,,
2012-08-02,1131.7,1126.5,-0.03,-0.04,-0.04,-0.04,-0.03,-0.03,2.0,8.0,9.0,9.0,9.0,1.75,2.50
2012-08-03,1134.8,1131.7,-0.02,-0.02,-0.07,-0.07,-0.02,-0.02,2.0,1.5,1.0,-5.0,-13.0,4.00,2.40
2012-08-06,1129.0,1134.8,0.01,0.01,0.00,0.00,0.01,0.01,-2.0,-4.5,-5.0,-6.0,-5.0,-9.50,2.40
2012-08-07,1128.8,1129.0,-0.01,0.00,0.00,0.00,-0.01,-0.01,1.0,1.5,0.0,-8.0,-10.0,0.50,2.50
2012-08-08,1128.3,1128.8,-0.01,-0.02,-0.02,-0.02,-0.01,-0.01,0.0,-2.0,-2.0,-4.0,-7.0,1.25,2.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-25,1313.7,1313.0,-0.02,-0.06,-0.07,-0.09,-0.01,-0.01,-4.0,-1.0,0.0,-2.0,0.0,3.35,-0.60
2022-07-26,1307.6,1313.7,0.01,-0.03,-0.02,-0.03,0.01,0.01,2.0,-1.0,0.0,1.0,1.0,1.15,-0.70
2022-07-27,1313.3,1307.6,0.00,-0.03,-0.05,-0.06,0.02,0.02,2.0,4.0,5.0,5.0,5.0,-0.50,-0.55


In [4]:
all4.columns

Index(['종가_ex', '전일종가_ex', '대비_국고1년', '대비_국고3년', '대비_국고5년', '대비_국고10년',
       '대비_통안1년', '대비_통안2년', '대비_베이시스1년', '대비_베이시스2년', '대비_베이시스3년',
       '대비_베이시스5년', '대비_베이시스10년', '대비_ndf', '스왑포인트_1월물'],
      dtype='object')

In [18]:
df_Target = all4["종가_ex"]
df_data = all4[['전일종가_ex', '대비_국고3년', '대비_국고5년', '대비_베이시스1년',
       '대비_베이시스10년', '대비_ndf']]

In [19]:
df_data.head(3)

,전일종가_ex,대비_국고3년,대비_국고5년,대비_베이시스1년,대비_베이시스10년,대비_ndf
DateTime,,,,,,
2012-08-02,1126.5,-0.04,-0.04,2.0,9.0,1.75
2012-08-03,1131.7,-0.02,-0.07,2.0,-13.0,4.00
2012-08-06,1134.8,0.01,0.00,-2.0,-5.0,-9.50


In [16]:

df_Data=df_Data.dropna(inplace=True)

C:\Users\hyeok\AppData\Local\Temp\ipykernel_30728\1996261187.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Data=df_Data.dropna(inplace=True)


In [20]:
import statsmodels.api as sm
feature_add = sm.add_constant(df_data, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 4.120e+04
Date:                Sun, 21 Aug 2022   Prob (F-statistic):               0.00
Time:                        16:23:32   Log-Likelihood:                -7675.5
No. Observations:                2455   AIC:                         1.536e+04
Df Residuals:                    2448   BIC:                         1.541e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.5370      2.279      1.552      0.121      -0.932       8.006
전일종가_ex        0.9969      0.002    497.025      0.000       0.993       1.001
대비_국고3년       -5.0050      1.981     -2.526      0.012      -8.890      -1.120
대비_국고5년       10.3078      3.409      3.024      0.003       3.623      16.993
대비_베이시스1년     -0.2529      0.025    -10.069      0.000      -0.302      -0.204
대비_베이시스10년     0.0426      0.024      1.747      0.081      -0.005       0.090
대비_ndf         0.0273      0.005      5.212      0.000       0.017       0.038
==============================================================================
Omnibus:                       58.420   Durbin-Watson:                   2.139
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              130.969
Skew:                           0.033   Prob(JB):                     3.63e-29
Kurtosis:                       4.130   Cond. No.                     3.61e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.61e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
X_train, X_test, y_train, y_test = train_test_split(feature_add, df_Target,test_size=0.2, random_state=30 )

lr = LinearRegression()
lr.fit (X_train, y_train)
y_preds = lr.predict(X_test)
mae = mean_absolute_error(y_test,y_preds)
rmse = np.sqrt(mae)
rmse

2.1395541961871594

In [25]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
X_train = df_data
def feature_engineering_XbyVIF(X_train):
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(X_train.values,i)
                         for i in range(X_train.shape[1])]
    vif["Feature"] = X_train.columns
    return vif
vif = feature_engineering_XbyVIF(X_train)
print(vif)

   VIF_Factor     Feature
0    1.000030     전일종가_ex
1    1.214760     대비_국고3년
2    1.283296     대비_국고5년
3    1.661990   대비_베이시스1년
4    1.744664  대비_베이시스10년
5    1.002244      대비_ndf
